#### ***Imports***

In [1]:
# --- Bloque 0: imports y opciones ---
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

#### ***Rutas de ficheros***

In [2]:
# --- Bloque 1: rutas ---
PATH_RRHH         = "rrhh_turno.csv"              # año_mes, horas_teoricas, reduccion_tco, horas_ajustadas, horas_enfermedad, horas_accidente, horas_permiso, horas_netas
PATH_INGENIERIA   = "datos_referencias.csv"       # ref_id, familia, peso_neto_kg
PATH_MOV_ALMACEN  = "almacen_movimientos.csv"     # mov_id, material_lot_id, item_ref_id, tipo_mov, qty, fecha_ts
PATH_COMPRAS      = "compras_lotes.csv"           # material_lot_id, laminado_id, ref_materia, qty_recibida, udm, peso_bruto, uds, fecha_recepcion_ts
PATH_WORK_ORDERS  = "ordenes_header.csv"          # work_order_id, ref_id, familia, cliente, qty_plan, fecha_lanzamiento, due_date, planta_inicio
PATH_HIST_OPS     = "produccion_operaciones.csv"  # work_order_id,work_order_id_raw,op_id,machine_id,machine_name,planta,op_text,ref_id,ref_id_raw,ts_ini,ts_fin,duracion_min,piezas_ok,piezas_scrap,evento,tipo_incidencia,operario_id,operario_nombre,of_impute_src,ref_impute_src,origen_fichero,origen_ruta,material_lot_id
OUTPUT_PATH       = "dataset_integrado_2025.csv"  # salida integrada

#### ***Limpieza y Helpers***

In [3]:
# --- Bloque 2: helpers de limpieza ---

def norm_ref(x, width=6):
    """
    Normaliza referencias:
    - Acepta strings, ints, floats o NaN.
    - Devuelve string left-pad con ceros a 'width' (por defecto 6).
    """
    if pd.isna(x):
        return np.nan
    # casos tipo 081901, '081901', 81901.0, 81000.0
    try:
        # Si viene como '000012', respeta
        s = str(x).strip()
        # Quita '.0' si es float representado como string
        if s.endswith(".0"):
            s = s[:-2]
        # Si contiene no-dígitos, elimina espacios y vuelve a probar
        if not s.isdigit():
            # intenta como float->int
            v = int(round(float(s)))
            return str(v).zfill(width)
        return s.zfill(width)
    except:
        try:
            v = int(round(float(x)))
            return str(v).zfill(width)
        except:
            return np.nan

def to_month_str(ts):
    """Convierte timestamp a 'YYYY-MM' string."""
    if pd.isna(ts):
        return np.nan
    return f"{ts.year:04d}-{ts.month:02d}"

def coerce_dt(s):
    """to_datetime tolerante, NaT si no se puede (e.g., 'PENDING')."""
    return pd.to_datetime(s, errors="coerce")

#### ***Carga CSV's***

In [4]:
# --- Bloque 3: carga segura de CSVs ---

rrhh = pd.read_csv(PATH_RRHH, dtype=str)
ing  = pd.read_csv(PATH_INGENIERIA, dtype=str)
movs = pd.read_csv(PATH_MOV_ALMACEN, dtype=str)
compr= pd.read_csv(PATH_COMPRAS, dtype=str)
wo   = pd.read_csv(PATH_WORK_ORDERS, dtype=str)
ops  = pd.read_csv(PATH_HIST_OPS, dtype=str)

In [7]:
# --- Bloque 4.1: RRHH mensual (robusto) ---
# Limpieza de cabeceras: espacios, BOM, guiones, mayúsculas
rrhh.columns = (rrhh.columns
                .str.strip()
                .str.replace('\ufeff','', regex=False)
                .str.replace(' ', '_')
                .str.replace('-', '_')
                .str.replace(r'[^0-9A-Za-z_ñáéíóúÑÁÉÍÓÚ]', '', regex=True)
                .str.lower())

# Normaliza variantes comunes
rrhh = rrhh.rename(columns={
    'ano_mes': 'año_mes',
    'anio_mes': 'año_mes',
    'mes_anio': 'año_mes',
    'horas_teoricas_hrs': 'horas_teoricas',
    'horas_teoricashrs': 'horas_teoricas',
})

expected = [
    "año_mes","horas_teoricas","reduccion_tco","horas_ajustadas",
    "horas_enfermedad","horas_accidente","horas_permiso","horas_netas"
]

# Crea columnas faltantes para evitar KeyError
for c in expected:
    if c not in rrhh.columns:
        rrhh[c] = np.nan

# Tipos
rrhh["año_mes"] = rrhh["año_mes"].astype(str).str.strip()
for c in ["horas_teoricas","reduccion_tco","horas_ajustadas","horas_enfermedad","horas_accidente","horas_permiso","horas_netas"]:
    rrhh[c] = pd.to_numeric(rrhh[c], errors="coerce")

rrhh = rrhh.drop_duplicates(subset=["año_mes"])

print("RRHH columnas detectadas:", list(rrhh.columns))

RRHH columnas detectadas: ['año_mes', 'horas_teoricas', 'reduccion_tco', 'horas_ajustadas', 'horas_enfermedad', 'horas_accidente', 'horas_permiso', 'horas_netas']


#### ***Limpieza por tabla***

In [8]:
# --- Bloque 4.1: RRHH mensual ---
rrhh["año_mes"] = rrhh["año_mes"].astype(str).str.strip()
cols_num = ["horas_teoricas","reduccion_tco","horas_ajustadas","horas_enfermedad","horas_accidente","horas_permiso","horas_netas"]
for c in cols_num:
    rrhh[c] = pd.to_numeric(rrhh[c], errors="coerce")
rrhh = rrhh.drop_duplicates(subset=["año_mes"])

# --- Bloque 4.2: Ingeniería (maestro de piezas) ---
ing["ref_id_str"] = ing["ref_id"].map(norm_ref)
ing["familia"] = ing["familia"].astype(str).str.strip()
ing["peso_neto_kg"] = pd.to_numeric(ing["peso_neto_kg"], errors="coerce")

# --- Bloque 4.3: Movimientos a almacén (solo IN) ---
movs["fecha_ts"] = coerce_dt(movs["fecha_ts"])
movs = movs[movs["tipo_mov"].str.upper().eq("IN")]
movs["item_ref_id_str"] = movs["item_ref_id"].map(norm_ref)
movs["qty"] = pd.to_numeric(movs["qty"], errors="coerce").fillna(0)
movs["fecha"] = movs["fecha_ts"].dt.normalize()  # fecha sin hora
mov_aggr = (movs
            .groupby(["item_ref_id_str","fecha"], as_index=False)["qty"]
            .sum()
            .rename(columns={"qty":"qty_in_almacen_dia"}))

# --- Bloque 4.4: Compras de materia prima (lotes) ---
compr["fecha_recepcion_ts"] = coerce_dt(compr["fecha_recepcion_ts"])
compr["material_lot_id"] = compr["material_lot_id"].astype(str).str.strip()
compr["ref_materia_str"] = compr["ref_materia"].map(norm_ref)
for c in ["qty_recibida","peso_bruto","uds"]:
    if c in compr:
        compr[c] = pd.to_numeric(compr[c], errors="coerce")

# --- Bloque 4.5: Work orders ---
wo["work_order_id"] = wo["work_order_id"].astype(str).str.strip()
wo["ref_id_str"] = wo["ref_id"].map(norm_ref)
wo["qty_plan"] = pd.to_numeric(wo["qty_plan"], errors="coerce")
for c in ["fecha_lanzamiento","due_date"]:
    wo[c] = coerce_dt(wo[c])
wo["planta_inicio"] = wo["planta_inicio"].replace({"PENDING":np.nan}).astype("string")

# --- Bloque 4.6: Histórico de operaciones ---
ops["ts_ini"] = coerce_dt(ops["ts_ini"])
ops["ts_fin"] = coerce_dt(ops["ts_fin"])
ops["duracion_min"] = pd.to_numeric(ops["duracion_min"], errors="coerce")
ops["piezas_ok"] = pd.to_numeric(ops["piezas_ok"], errors="coerce").fillna(0)
ops["piezas_scrap"] = pd.to_numeric(ops["piezas_scrap"], errors="coerce").fillna(0)
# ref preferente: 'ref_id' si existe, si no 'ref_id_raw'
ops["ref_id_str"] = np.where(ops.get("ref_id").notna(), ops["ref_id"].map(norm_ref), ops.get("ref_id_raw").map(norm_ref))
# fecha de fin para agregados y joins diarios
ops["fecha"] = ops["ts_fin"].dt.normalize()
# downtime solo cuando evento=Incidencia
ops["evento"] = ops["evento"].astype(str)
ops["downtime_min"] = np.where(ops["evento"].str.lower().str.contains("incidencia"), ops["duracion_min"], 0).astype(float)
# material_lot_id a string y 0 -> NaN
if "material_lot_id" in ops:
    ops["material_lot_id"] = ops["material_lot_id"].astype(str).str.strip()
    ops.loc[ops["material_lot_id"].isin(["0","0.0","nan","None",""]), "material_lot_id"] = np.nan

/tmp/ipykernel_4083/2875816976.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  wo["planta_inicio"] = wo["planta_inicio"].replace({"PENDING":np.nan}).astype("string")


#### ***Unión Base Sobre Operaciones***

In [9]:
# --- Bloque 5: joins principales sobre ops ---

# 5.1 Join Ingeniería (familia, peso)
ops1 = ops.merge(ing[["ref_id_str","familia","peso_neto_kg"]], on="ref_id_str", how="left")

# 5.2 Join Work Orders (cliente, qty_plan, due_date, planta_inicio)
ops2 = ops1.merge(
    wo[["work_order_id","ref_id_str","cliente","qty_plan","fecha_lanzamiento","due_date","planta_inicio"]],
    on=["work_order_id","ref_id_str"],
    how="left"
)

# 5.3 Join Compras por material_lot_id (si existe)
if "material_lot_id" in ops2.columns:
    ops3 = ops2.merge(
        compr[["material_lot_id","ref_materia_str","qty_recibida","peso_bruto","uds","fecha_recepcion_ts"]],
        on="material_lot_id",
        how="left"
    )
else:
    ops3 = ops2.copy()

# 5.4 Join Movimientos IN agregados por ref y fecha (qty_in_almacen_dia)
ops4 = ops3.merge(
    mov_aggr.rename(columns={"item_ref_id_str":"ref_id_str"}),
    on=["ref_id_str","fecha"],
    how="left"
)

# 5.5 Join RRHH por mes de ts_fin (global, sin planta)
ops4["año_mes"] = ops4["ts_fin"].dt.strftime("%Y-%m")
ops5 = ops4.merge(rrhh, on="año_mes", how="left")

#### ***Métricas Derivadas***

In [10]:
# --- Bloque 6: features derivadas fáciles ---

total_pzs = ops5["piezas_ok"] + ops5["piezas_scrap"]
ops5["throughput_uph"] = np.where(ops5["duracion_min"]>0, 60.0 * ops5["piezas_ok"] / ops5["duracion_min"], np.nan)
ops5["scrap_rate"] = np.where(total_pzs>0, ops5["piezas_scrap"] / total_pzs, np.nan)
ops5["consumo_materia_kg"] = ops5["peso_neto_kg"] * ops5["piezas_ok"]

#### ***Lead time a stock (fin de operación → siguiente entrada IN)***

In [12]:
# --- Bloque 7: lead_time_al_almacen_dias (robusto, por ref) ---

# 1) Preparar tablas de fechas
ends = ops5[["ref_id_str","ts_fin"]].dropna().copy()
ends["end_date"] = ends["ts_fin"].dt.normalize()
ends = ends.dropna(subset=["ref_id_str","end_date"])

mov_dates = mov_aggr[["item_ref_id_str","fecha"]].drop_duplicates().rename(columns={"item_ref_id_str":"ref_id_str"})
mov_dates = mov_dates.dropna(subset=["ref_id_str","fecha"])

# Garantizar tipos datetime
ends["end_date"] = pd.to_datetime(ends["end_date"])
mov_dates["fecha"] = pd.to_datetime(mov_dates["fecha"])

if mov_dates.empty:
    # Sin movimientos: no se puede calcular lead time
    ops5["end_date"] = ops5["ts_fin"].dt.normalize()
    ops5["lead_time_al_almacen_dias"] = np.nan
else:
    # 2) Índice de movimientos por ref, ordenado
    mov_group = {k: v.sort_values("fecha")["fecha"].values
                 for k, v in mov_dates.groupby("ref_id_str")}

    # 3) Para cada referencia, primer movimiento >= end_date
    def compute_lead_for_ref(sub):
        ref = sub.name
        end_vals = sub["end_date"].values
        stock_vals = mov_group.get(ref, None)
        if stock_vals is None or len(stock_vals) == 0:
            return pd.Series([np.nan]*len(sub), index=sub.index)
        idx = np.searchsorted(stock_vals, end_vals, side="left")
        chosen = []
        for i in idx:
            if i >= len(stock_vals):
                chosen.append(pd.NaT)
            else:
                chosen.append(stock_vals[i])
        chosen = pd.to_datetime(chosen)
        return (chosen - sub["end_date"]).dt.days.astype("float")

    lead_series = ends.groupby("ref_id_str", group_keys=False).apply(compute_lead_for_ref)
    ends["lead_time_al_almacen_dias"] = lead_series.values

    # 4) Mapear de vuelta
    key = ends["ref_id_str"].astype(str) + "|" + ends["end_date"].astype(str)
    lead_map = dict(zip(key, ends["lead_time_al_almacen_dias"]))

    ops5["end_date"] = ops5["ts_fin"].dt.normalize()
    ops5["__k"] = ops5["ref_id_str"].astype(str) + "|" + ops5["end_date"].astype(str)
    ops5["lead_time_al_almacen_dias"] = ops5["__k"].map(lead_map)
    ops5 = ops5.drop(columns=["__k"])

/tmp/ipykernel_4083/406989896.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lead_series = ends.groupby("ref_id_str", group_keys=False).apply(compute_lead_for_ref)


#### ***Fusión Dataset***

In [14]:
# --- Bloque 8: orden final de columnas útiles ---

cols_final = [
    # claves y tiempo
    "work_order_id","op_id","machine_id","machine_name","planta","op_text",
    "ref_id_str","familia","peso_neto_kg","material_lot_id","ref_materia_str",
    "ts_ini","ts_fin","fecha","duracion_min","evento","tipo_incidencia",
    # cantidades
    "piezas_ok","piezas_scrap","qty_plan","qty_in_almacen_dia",
    # rrhh
    "año_mes","horas_teoricas","reduccion_tco","horas_ajustadas","horas_enfermedad","horas_accidente","horas_permiso","horas_netas",
    # compras
    "qty_recibida","peso_bruto","uds","fecha_recepcion_ts",
    # derivados
    "throughput_uph","scrap_rate","downtime_min","consumo_materia_kg","lead_time_al_almacen_dias",
]

# Mantén solo las que existan
cols_final = [c for c in cols_final if c in ops5.columns]
df = ops5[cols_final].copy()

# Orden y tipos suaves
num_cols = ["duracion_min","piezas_ok","piezas_scrap","qty_plan","qty_in_almacen_dia",
            "horas_teoricas","reduccion_tco","horas_ajustadas","horas_enfermedad","horas_accidente","horas_permiso","horas_netas",
            "qty_recibida","peso_bruto","uds","throughput_uph","scrap_rate","downtime_min","consumo_materia_kg","peso_neto_kg","lead_time_al_almacen_dias"]
for c in num_cols:
    if c in df:
        df[c] = pd.to_numeric(df[c], errors="coerce")

#### ***Tests Finales***

In [15]:
# --- Bloque 9: sanity checks mínimos ---
print("Filas integradas:", len(df))
print("Refs distintas:", df["ref_id_str"].nunique() if "ref_id_str" in df else "-")
print("Máquinas distintas:", df["machine_name"].nunique() if "machine_name" in df else "-")
print(df.head(3))

Filas integradas: 362433
Refs distintas: 91
Máquinas distintas: 78
  work_order_id         op_id machine_id machine_name    planta       op_text ref_id_str             familia  peso_neto_kg material_lot_id ref_materia_str  \
0       24/0658  SOLDADURA-RE        515    SOLDADORA  Zaldibar  SOLDADURA-RE     081000  CORONA DE ARRANQUE          1.59             NaN             NaN   
1       24/0658  SOLDADURA-RE        515    SOLDADORA  Zaldibar  SOLDADURA-RE     081000  CORONA DE ARRANQUE          1.59             NaN             NaN   
2       24/0674   RECTIFICADO       1001  Linea Luk 1   Abadiño   RECTIFICADO     124203                 NaN           NaN             NaN             NaN   

               ts_ini              ts_fin      fecha  duracion_min      evento  tipo_incidencia  piezas_ok  piezas_scrap  qty_plan  qty_in_almacen_dia  \
0 2025-01-06 21:47:00 2025-01-06 21:50:00 2025-01-06           3.0  Producción              NaN          0             0    4008.0                

#### ***Guardado Dataset***

In [16]:
# --- Bloque 10: export ---
df.to_csv(OUTPUT_PATH, index=False)
print("Guardado en:", OUTPUT_PATH)

Guardado en: dataset_integrado_2025.csv


In [18]:
# --- Bloque 11: listado de máquinas y top-5 por recurso ---

# 11.1 Máquinas distintas
maquinas = (df[["machine_id","machine_name","planta"]]
            .drop_duplicates()
            .sort_values(["planta","machine_name"], na_position="last"))
print("Máquinas distintas:", len(maquinas))
print(maquinas.head(20))

Máquinas distintas: 84
       machine_id         machine_name   planta
195210        134         Curvadora134  Abadiño
1390           25          Curvadora25  Abadiño
1640          122    Fresadora Hey 122  Abadiño
1650           12          Fresadora12  Abadiño
5628          140         Fresadora140  Abadiño
29423         141         Fresadora141  Abadiño
1619           80          Fresadora80  Abadiño
1503          110         Granalladora  Abadiño
11            517  Granalladora Sthick  Abadiño
1626          118                Horno  Abadiño
7             519         Horno Arrola  Abadiño
2            1001          Linea Luk 1  Abadiño
16           1002          Linea Luk 2  Abadiño
108          1003          Linea Luk 3  Abadiño
1641         4001          Linea SACHS  Abadiño
5187         3001     Linea Volkswagen  Abadiño
436            32               PRENSA  Abadiño
1399           31               PRENSA  Abadiño
2966           27             Prensa27  Abadiño
20230          95

In [20]:
# --- Bloque 12: resumen jerárquico OF -> ref -> piezas ---

# 12.1 Agregado principal
of_ref = (df.groupby(["work_order_id","ref_id_str","familia"], as_index=False)
            .agg(piezas_ok=("piezas_ok","sum"),
                 piezas_scrap=("piezas_scrap","sum"),
                 qty_plan=("qty_plan","max"))
         )
of_ref["total"] = of_ref["piezas_ok"] + of_ref["piezas_scrap"]
of_ref["scrap_rate"] = np.where(of_ref["total"]>0, of_ref["piezas_scrap"]/of_ref["total"], np.nan)
of_ref["cumplimiento"] = np.where(of_ref["qty_plan"].notna() & (of_ref["qty_plan"]>0),
                                  of_ref["piezas_ok"]/of_ref["qty_plan"], np.nan)

print("OF->ref resumen (primeras 10 filas):")
print(of_ref.head(100))

# 12.2 Vista rápida por OF concreto (cambia el ID)
OF_EJEMPLO = of_ref["work_order_id"].dropna().unique()[0] if len(of_ref)>0 else None
if OF_EJEMPLO:
    print(f"\nDetalle para OF {OF_EJEMPLO}:")
    print(of_ref[of_ref["work_order_id"]==OF_EJEMPLO]
          .sort_values("piezas_ok", ascending=False)
          [["work_order_id","ref_id_str","familia","piezas_ok","piezas_scrap","total","scrap_rate","qty_plan","cumplimiento"]]
         )

OF->ref resumen (primeras 10 filas):
   work_order_id ref_id_str              familia  piezas_ok  piezas_scrap  qty_plan   total  scrap_rate  cumplimiento
0        22/1009     003501   CORONA DE ARRANQUE        868            31     110.0     899    0.034483      7.890909
1        23/0172     191000   CORONA DE ARRANQUE          0             0       NaN       0         NaN           NaN
2        23/0366     818003  INKREMENT GEBERRING       9466            14       NaN    9480    0.001477           NaN
3        23/0378     010002   CORONA DE ARRANQUE        556            16       NaN     572    0.027972           NaN
4        23/0554     002260   CORONA DE ARRANQUE         60             0       NaN      60    0.000000           NaN
..           ...        ...                  ...        ...           ...       ...     ...         ...           ...
95       25/0049     904802   CORONA DE ARRANQUE     109380            92    4068.0  109472    0.000840     26.887906
96       25/0051   